# Finetune Your Chatbot on a Single Node Xeon SPR 

NeuralChat is a customizable chat framework designed to create user own chatbot within few minutes on multiple architectures. This notebook will introduce how to finetune your chatbot on the customized data on a single node Xeon SPR.

## Prepare Environment

Install intel extension for transformers:

In [ ]:
!pip install intel-extension-for-transformers

Install Requirements:

In [ ]:
!git clone https://github.com/intel/intel-extension-for-transformers.git

In [ ]:
%cd ./intel-extension-for-transformers/intel_extension_for_transformers/neural_chat/
!pip install -r requirements.txt
%cd ../../../

## Prepare the Dataset
We select 3 kind of datasets to conduct the finetuning process for different tasks.

1. Text Generation (General domain instruction): We use the [Alpaca dataset](https://github.com/tatsu-lab/stanford_alpaca) from Stanford University as the general domain dataset to fine-tune the model. This dataset is provided in the form of a JSON file, [alpaca_data.json](https://github.com/tatsu-lab/stanford_alpaca/blob/main/alpaca_data.json). In Alpaca, researchers have manually crafted 175 seed tasks to guide `text-davinci-003` in generating 52K instruction data for diverse tasks.

2. Summarization: An English-language dataset [cnn_dailymail](https://huggingface.co/datasets/cnn_dailymail) containing just over 300k unique news articles as written by journalists at CNN and the Daily Mail, is used for this task.

3. Code Generation: To enhance code performance of LLMs (Large Language Models), we use the [theblackcat102/evol-codealpaca-v1](https://huggingface.co/datasets/theblackcat102/evol-codealpaca-v1).



## Finetune Your Chatbot

We employ the [LoRA approach](https://arxiv.org/pdf/2106.09685.pdf) to finetune the LLM efficiently.

Finetune the model on Alpaca-format dataset to conduct text generation:

In [14]:
import json

# Path to the JSON file
json_file_path = "/home/u10bbfacac939571647f57e237c7e0f7/intel-extension-for-transformers/intel_extension_for_transformers/neural_chat/docs/notebooks/stanford_alpaca-main/alpaca_data.json"

try:
    with open(json_file_path, 'r') as file:
        data = json.load(file)
    print("The JSON file is valid.")
except json.JSONDecodeError as e:
    print(f"Invalid JSON file: {e}")
except FileNotFoundError:
    print(f"The file {json_file_path} does not exist.")
import logging

logging.basicConfig(level=logging.DEBUG)

# Your script here

The JSON file is valid.


In [2]:
from transformers import TrainingArguments
from intel_extension_for_transformers.neural_chat.config import (
    ModelArguments,
    DataArguments,
    FinetuningArguments,
    TextGenerationFinetuningConfig,
)
from intel_extension_for_transformers.neural_chat.chatbot import finetune_model
model_args = ModelArguments(model_name_or_path="https://huggingface.co/meta-llama/Llama-2-7b-chat-hf")
data_args = DataArguments(train_file="alpaca_data.json", validation_split_percentage=1)
training_args = TrainingArguments(
    output_dir='./tmp',
    do_train=True,
    do_eval=True,
    num_train_epochs=3,
    overwrite_output_dir=True,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    gradient_accumulation_steps=2,
    save_strategy="no",
    log_level="info",
    save_total_limit=2,
    bf16=True,
)
finetune_args = FinetuningArguments()
finetune_cfg = TextGenerationFinetuningConfig(
            model_args=model_args,
            data_args=data_args,
            training_args=training_args,
            finetune_args=finetune_args,
        )
finetune_model(finetune_cfg)

[INFO|training_args.py:2048] 2024-07-14 03:04:46,375 >> PyTorch: setting up devices
[INFO|training_args.py:1751] 2024-07-14 03:04:46,380 >> The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
[WARNING|_logger.py:72] 2024-07-14 03:04:46,397 >> Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
[INFO|training_args.py:2048] 2024-07-14 03:04:46,400 >> PyTorch: setting up devices
2024-07-14 03:04:46,403 - _logger.py - intel_extension_for_transformers.transformers.llm.finetuning.finetuning - WARNING - Process rank: 0, device: cpu
distributed training: True, 16-bits training: True
2024-07-14 03:04:46,406 - finetuning.py - intel_extensi

(…)ingface.co/meta-llama/Llama-2-7b-chat-hf:   0%|          | 0.00/241k [00:00<?, ?B/s]

2024-07-14 03:04:46,692 - chatbot.py - intel_extension_for_transformers.neural_chat.chatbot - ERROR - Exception: It looks like the config file at '/tmp/tmp2sro_r9a' is not a valid JSON file.
2024-07-14 03:04:46,694 - error_utils.py - intel_extension_for_transformers.neural_chat.utils.error_utils - ERROR - neuralchat error: LORA finetuning failed


Finetune the model on the summarization task:

In [24]:
from transformers import TrainingArguments
from intel_extension_for_transformers.neural_chat.config import (
    ModelArguments,
    DataArguments,
    FinetuningArguments,
    TextGenerationFinetuningConfig,
)
from intel_extension_for_transformers.neural_chat.chatbot import finetune_model
model_args = ModelArguments(model_name_or_path="https://huggingface.co/meta-llama/Llama-2-7b-chat-hf")
data_args = DataArguments(dataset_name="cnn_dailymail", dataset_config_name="3.0.0")
training_args = TrainingArguments(
    output_dir='./tmp',
    do_train=True,
    do_eval=True,
    num_train_epochs=3,
    overwrite_output_dir=True,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    gradient_accumulation_steps=2,
    save_strategy="no",
    log_level="info",
    save_total_limit=2,
    bf16=True
)
finetune_args = FinetuningArguments(task='summarization')
finetune_cfg = TextGenerationFinetuningConfig(
            model_args=model_args,
            data_args=data_args,
            training_args=training_args,
            finetune_args=finetune_args,
        )
finetune_model(finetune_cfg)

[INFO|training_args.py:2048] 2024-07-13 15:31:44,885 >> PyTorch: setting up devices
[INFO|training_args.py:1751] 2024-07-13 15:31:44,888 >> The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
[WARNING|_logger.py:72] 2024-07-13 15:31:44,913 >> Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
[INFO|training_args.py:2048] 2024-07-13 15:31:44,915 >> PyTorch: setting up devices
2024-07-13 15:31:44,918 - _logger.py - intel_extension_for_transformers.transformers.llm.finetuning.finetuning - WARNING - Process rank: 0, device: cpu
distributed training: True, 16-bits training: True
2024-07-13 15:31:44,928 - finetuning.py - intel_extensi

(…)ingface.co/meta-llama/Llama-2-7b-chat-hf:   0%|          | 0.00/241k [00:00<?, ?B/s]

2024-07-13 15:31:45,171 - chatbot.py - intel_extension_for_transformers.neural_chat.chatbot - ERROR - Exception: It looks like the config file at '/tmp/tmpuoq5w14z' is not a valid JSON file.
2024-07-13 15:31:45,173 - error_utils.py - intel_extension_for_transformers.neural_chat.utils.error_utils - ERROR - neuralchat error: LORA finetuning failed


Finetune the model on the code generation task:

In [1]:
from transformers import TrainingArguments
from intel_extension_for_transformers.neural_chat.config import (
    ModelArguments,
    DataArguments,
    FinetuningArguments,
    TextGenerationFinetuningConfig,
)
from intel_extension_for_transformers.neural_chat.chatbot import finetune_model
model_args = ModelArguments(model_name_or_path="https://huggingface.co/meta-llama/Llama-2-7b-chat-hf")
data_args = DataArguments(dataset_name="theblackcat102/evol-codealpaca-v1")
training_args = TrainingArguments(
    output_dir='./tmp',
    do_train=True,
    do_eval=True,
    num_train_epochs=3,
    overwrite_output_dir=True,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    gradient_accumulation_steps=2,
    save_strategy="no",
    log_level="info",
    save_total_limit=2,
    bf16=True
)
finetune_args = FinetuningArguments(task='code-generation')
finetune_cfg = TextGenerationFinetuningConfig(
            model_args=model_args,
            data_args=data_args,
            training_args=training_args,
            finetune_args=finetune_args,
        )
finetune_model(finetune_cfg)

2024-07-14 03:03:50.881932: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-07-14 03:03:51.550436: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:479] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-14 03:03:51.725435: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:10575] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-14 03:03:51.725479: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1442] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-14 03:03:51.880080: I tensorflow/core/platform/cpu_feature_gua

(…)ingface.co/meta-llama/Llama-2-7b-chat-hf:   0%|          | 0.00/241k [00:00<?, ?B/s]

2024-07-14 03:04:01,861 - chatbot.py - intel_extension_for_transformers.neural_chat.chatbot - ERROR - Exception: It looks like the config file at '/tmp/tmpkdpiewfk' is not a valid JSON file.
2024-07-14 03:04:01,863 - error_utils.py - intel_extension_for_transformers.neural_chat.utils.error_utils - ERROR - neuralchat error: LORA finetuning failed
